# Scenario 1

In [1]:
import pandas as pd
import altair as alt
import numpy as np

In [2]:
def read_data(versuch:int):
    tx_logs = pd.read_csv(f"../logs/s1_v{versuch}_transmitter.csv", sep=';')
    rx_logs = pd.read_csv(f"../logs/s1_v{versuch}_receiver.csv", sep=';')
    
    tx_logs["ROLE"] = ["Tx"]*tx_logs.shape[0]
    rx_logs["ROLE"] = ["Rx"]*rx_logs.shape[0]
    
    return tx_logs.append(rx_logs, ignore_index=True)

## Experiment N

In [3]:
experiment = 1
logs = read_data(experiment)
tx_logs = logs[logs["ROLE"]=="Tx"]
rx_logs = logs[logs["ROLE"]=="Rx"]

C:\Users\Konst\AppData\Local\Temp\ipykernel_11632\3360409294.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return tx_logs.append(rx_logs, ignore_index=True)


### Round Trip Times (RTT)

In [4]:
RTTs = pd.DataFrame((tx_logs[tx_logs["ACK"]==240].set_index("SEQ")["TIMESTAMP"] - tx_logs[tx_logs["ACK"]==0].set_index("SEQ")["TIMESTAMP"]))
RTTs.columns=["RTT"]

In [5]:
alt.Chart(RTTs.reset_index(), title=f"Round Trip Times in Scenario 1 Experiment {experiment}").mark_bar().encode(
    x="SEQ:O",
    y=alt.Y("RTT:Q", title="RTT (ms)")
)

alt.Chart(...)

### RSSI

In [6]:
RSSI_df = logs.loc[((logs["ACK"]==0) & (logs["ROLE"]=="Rx")) | ((logs["ACK"]==240) & (logs["ROLE"]=="Tx"))][["SEQ", "RSSI", "ROLE"]]

alt.Chart(RSSI_df, title=f"RSSI Values in Scenario 1 Experiment {experiment}").mark_point(filled=True).encode(
    x="SEQ:O",
    y=alt.Y("RSSI:Q", scale=alt.Scale(domain=(min(RSSI_df["RSSI"])-10, max(RSSI_df["RSSI"])+10))),
    color="ROLE:N"
)

alt.Chart(...)

In [7]:
alt.Chart(RSSI_df, title=f"RSSI Variances Scenario 1 Experiment {experiment}").mark_bar().encode(
    x="ROLE:N",
    y="variance(RSSI)",
    color="ROLE:N"
)

alt.Chart(...)

### LQI

In [8]:
LQI_df = logs.loc[((logs["ACK"]==0) & (logs["ROLE"]=="Rx")) | ((logs["ACK"]==240) & (logs["ROLE"]=="Tx"))][["SEQ", "LQI", "ROLE"]]

alt.Chart(LQI_df, title=f"LQI Values in Scenario 1 Experiment {experiment}").mark_point(filled=True).encode(
    x="SEQ:O",
    y=alt.Y("LQI:Q", scale=alt.Scale(domain=(min(LQI_df["LQI"])-5, max(LQI_df["LQI"])+5))),
    color="ROLE:N"
)

alt.Chart(...)

In [9]:
alt.Chart(LQI_df, title=f"LQII Variances Scenario 1 Experiment {experiment}").mark_bar().encode(
    x="ROLE:N",
    y="variance(LQI)",
    color="ROLE:N"
)

alt.Chart(...)